In [169]:
%matplotlib inline
import pandas as pd
import numpy as np
tweets = pd.read_csv("data/Sentiment.csv")

In [170]:
tweets_clean = tweets[(tweets.relevant_yn_confidence > 0.8) & (tweets.candidate_confidence > 0.8) & (tweets.sentiment_confidence > 0.8) & (tweets.subject_matter_confidence > 0.8)]
tweets_clean = tweets_clean.drop(['id', 'candidate_confidence', 'relevant_yn', 'relevant_yn_confidence', 'sentiment_confidence', 'subject_matter_confidence', 'candidate_gold', 'relevant_yn_gold', 'sentiment_gold', 'subject_matter_gold', 'tweet_coord', 'tweet_id', 'user_timezone'], axis=1)
tweets_clean.head()

,candidate,sentiment,subject_matter,name,retweet_count,text,tweet_created,tweet_location
7,No candidate mentioned,Neutral,None of the above,RaulAReyes,0,Going on #MSNBC Live with @ThomasARoberts arou...,2015-08-07 09:54:44 -0700,New York NY
10,Donald Trump,Negative,None of the above,jnjsmom,0,@JGreenDC @realDonaldTrump In all fairness #Bi...,2015-08-07 09:54:42 -0700,"Peoria, IL"
14,Scott Walker,Positive,None of the above,In_Related_News,215,RT @pattonoswalt: I loved Scott Walker as Mark...,2015-08-07 09:54:42 -0700,"San Diego, California"
20,Ted Cruz,Positive,None of the above,rickymcghee,6,"RT @ChuckNellis: Cruz has class &amp; truth, t...",2015-08-07 09:54:39 -0700,"fripp island,sc/ southeast ga"
26,Scott Walker,Negative,Abortion,mch7576,19,RT @TheBaxterBean: Scott Walker's Abortion Ban...,2015-08-07 09:54:38 -0700,USA


In [171]:
print (tweets_clean.columns)

Index(['candidate', 'sentiment', 'subject_matter', 'name', 'retweet_count',
       'text', 'tweet_created', 'tweet_location'],
      dtype='object')


In [172]:
candidates = tweets_clean['candidate'].unique()
candidates = [x for x in candidates if ((x != 'No candidate mentioned') & (len(tweets_clean[tweets_clean.candidate == x]) > 50))]

In [173]:
import seaborn as sb
candidate_info = pd.DataFrame(columns=['percent_positive'], index=candidates)
for candidate in candidates:
    sentiments = tweets_clean[(tweets_clean.candidate == candidate)]['sentiment']
    pos_count = len(sentiments[sentiments == 'Positive'])
    neg_count = len(sentiments[sentiments == 'Negative'])
    candidate_info.loc[candidate,'percent_positive'] = pos_count / (pos_count + neg_count)
candidate_info

,percent_positive
Donald Trump,0.211091
Scott Walker,0.123077
Ted Cruz,0.695035
Jeb Bush,0.028169
John Kasich,0.76087
Chris Christie,0.151515
Ben Carson,0.640449
Rand Paul,0.263158
Mike Huckabee,0.314286
Marco Rubio,0.565217


## Sentiment

In [210]:
sentiments = tweets[tweets.sentiment != 'Neutral'].loc[:,['sentiment', 'text']]
sentiments_test = sentiments[:2000]
sentiments = sentiments[2000:]

positive_words = []
positive = sentiments[sentiments.sentiment == 'Positive']['text']
for tweet in positive:
    words = [e.lower() for e in tweet.split() if len(e) >= 3]
    positive_words.extend(words)
    
negative_words = []
negative = sentiments[sentiments.sentiment == 'Negative']['text']
for tweet in negative:
    words = [e.lower() for e in tweet.split() if len(e) >= 3]
    negative_words.extend(words)

In [211]:
import nltk
words = positive_words + negative_words
word_frequency = nltk.FreqDist(words)
word_list = word_frequency.keys()

In [219]:
def features(tweets, positive):
    training_set = []
    for tweet in tweets:
        tweet_words = [e.lower() for e in tweet.split() if len(e) >= 3]
        features = []
        for index, word in enumerate(word_list):
            features.append(word in tweet_words)
        training_set.append((features, positive))
    return training_set

In [ ]:
training_set = features(positive, 'Positive') + features(negative, 'Negetive')

In [ ]:
len(training_set)

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)